Import statement

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import numpy as np
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import h5py

Load data Functions

In [2]:

def shuffle(X, Y):
	p = np.random.permutation(X.shape[0])
	return (X[p], Y[p])

def load_images(path): 

	image_files = [f for f in listdir(path) if isfile(join(path, f))]
	images_arr = []
	for i in range(len(image_files)):
		image_np = plt.imread(path + '/' + image_files[i])
		images_arr.append(image_np)
	
	smaple_image = images_arr[0]
	images = np.array(images_arr)	
	#images = np.zeros((len(images_arr), smaple_image.shape[0], sample_image.shape[1], sample_image.shape[2]))
	
	#for i in range(len(images_arr)):
	#	images[i] = images_arr[i]
	
	return images


def load_data(path):
	"""
	Given the path of the dataset root folder, loads all the images from
	the subdirectory
	Parameters
		path -- path to the root folder of the dataset
	Returns
		(X_train, Y_train, X_test, Y_test)
	"""
	
	if isfile("X_train.h5") and isfile("Y_train.h5") and isfile("X_test.h5") and isfile("Y_test.h5"):
		X_train = None
		Y_train = None
		X_test = None
		Y_test = None

		with h5py.File('X_train.h5', 'r') as hf:
			X_train = hf['X_train'][:]

		with h5py.File('Y_train.h5', 'r') as hf:
			Y_train = hf['Y_train'][:]

		with h5py.File('X_test.h5', 'r') as hf:
			X_test = hf['X_test'][:]

		with h5py.File('Y_test.h5', 'r') as hf:
			Y_test = hf['Y_test'][:]
		return (X_train, Y_train, X_test, Y_test)
	
	directories = [f for f in listdir(path) if not(isfile(join(path, f)))]
	labels = directories


	X = load_images(path+directories[0])
	Y = np.array([0 for i in range(X.shape[0])])

	for i in range(1, len(directories)):
		image = load_images(path+directories[i])
		X = np.concatenate((X, image))
		Y = np.concatenate((Y, np.array([j for j in range(image.shape[0])])))
		
	(X, Y) = shuffle(X, Y)
	(X, Y) = shuffle(X, Y)

	X_train = X[0 : int(X.shape[0] * 0.8)]
	Y_train = Y[0 : int(Y.shape[0] * 0.8)]

	X_test = X[int(X.shape[0] * 0.8) :]
	Y_test = Y[int(Y.shape[0] * 0.8) :]

	with h5py.File('X_train.h5', 'w') as hf:
		hf.create_dataset('X_train', data=X_train)
	
	with h5py.File('Y_train.h5', 'w') as hf:
		hf.create_dataset('Y_train', data=Y_train)
	
	with h5py.File('X_test.h5', 'w') as hf:
		hf.create_dataset('X_test', data=X_test)

	with h5py.File('Y_test.h5', 'w') as hf:
		hf.create_dataset('Y_test', data=Y_test)

	return (X_train, Y_train, X_test, Y_test)

load data

In [3]:
(X_train, Y_train, X_test, Y_test) = load_data('./data/')

In [4]:
X_train = (X_train.astype(np.float32) / 255)
X_test = (X_test.astype(np.float32) / 255)

model

In [8]:
model = models.Sequential()

model.add(layers.Conv2D(256, (3, 3), padding='same', input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]), activation='relu'))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

#model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
#model.add(layers.MaxPooling2D(pool_size=(1, 1)))
#model.add(layers.Dropout(0.25))

#model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
#model.add(layers.MaxPooling2D(pool_size=(1, 1)))
#model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(1, 1)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(units=6, activation='softmax'))



In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
print(X_train[0:40].shape)
hisotry = model.fit(X_train[0:10], Y_train[0:10], epochs = 10, validation_data = (X_test, Y_test))

(40, 384, 512, 3)
Train on 10 samples, validate on 506 samples
Epoch 1/10
